# Étape 1: preparation des données (Tokenization + Suppression des Stopwords)

**1. Tokenisation**

In [1]:
import nltk
# Télécharger les ressources de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

file_path = '/content/hotel_reviews_preprocessed.xlsx'
data = pd.read_excel(file_path)

data['Tokenized_Review'] = data['Normalized_Review'].apply(word_tokenize)

# Afficher les résultats
print(data[['Normalized_Review', 'Tokenized_Review']].head())


                                   Normalized_Review  \
0                      ممتاز النظافة والطاقم متعاون    
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

                                    Tokenized_Review  
0                  [ممتاز, النظافة, والطاقم, متعاون]  
1  [استثنائي, سهولة, إنهاء, المعاملة, في, الاستقب...  
2  [استثنائي, انصح, بأختيار, الاسويت, و, بالاخص, ...  
3  [استغرب, تقييم, الفندق, كخمس, نجوم, لا, شي, يس...  
4  [جيد, المكان, جميل, وهاديء, كل, شي, جيد, ونظيف...  


**2. Suppression des Stopwords**

In [3]:
# Télécharger Arabic stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
import nltk

# Importe des stopwords de NLTK
arabic_stopwords = set(stopwords.words('arabic'))

# Liste des stopwords supplémentaires
additional_stopwords = {
    'و', 'في', 'على', 'من', 'إلى', 'عن', 'هذا', 'هذه', 'ذلك', 'تلك',
    'هو', 'هي', 'أنا', 'أنت', 'أنتم', 'هم', 'هن', 'كان', 'كانت', 'لك',
    'له', 'لها', 'لنا', 'عند', 'عليه', 'عليها', 'فيها', 'بين', 'هناك',
    'هنا', 'إذا', 'لكن', 'أو', 'بل', 'ثم', 'إن', 'أن', 'ألا', 'إلا',
    'حتى', 'كل', 'أي', 'بعض', 'بعد', 'قبل', 'أمام', 'خلف', 'تحت', 'فوق',
    'مع', 'لذلك', 'ذلك', 'مازال', 'قد',
    'عندما', 'مثل', 'إليها', 'بها', 'بهم', 'منهم', 'عليهم', 'بكم', 'أيضا'
}
# Fusionner les listes
arabic_stopwords.update(additional_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in arabic_stopwords]

data['Filtered_Review'] = data['Tokenized_Review'].apply(remove_stopwords)

# Afficher les résultats
print(data[['Tokenized_Review', 'Filtered_Review']].head())


                                    Tokenized_Review  \
0                  [ممتاز, النظافة, والطاقم, متعاون]   
1  [استثنائي, سهولة, إنهاء, المعاملة, في, الاستقب...   
2  [استثنائي, انصح, بأختيار, الاسويت, و, بالاخص, ...   
3  [استغرب, تقييم, الفندق, كخمس, نجوم, لا, شي, يس...   
4  [جيد, المكان, جميل, وهاديء, كل, شي, جيد, ونظيف...   

                                     Filtered_Review  
0                  [ممتاز, النظافة, والطاقم, متعاون]  
1  [استثنائي, سهولة, إنهاء, المعاملة, الاستقبال, ...  
2  [استثنائي, انصح, بأختيار, الاسويت, بالاخص, غرف...  
3  [استغرب, تقييم, الفندق, كخمس, نجوم, شي, يستحق,...  
4  [جيد, المكان, جميل, وهاديء, شي, جيد, ونظيف, حو...  


# Étape  2: Génération des Embeddings avec FastText (Skip-Gram)

**1. Génération des Embeddings**

In [5]:
from gensim.models import FastText

# Entraîner un modèle FastText Skip-Gram
fasttext_sg_model = FastText(data['Tokenized_Review'], vector_size=100, window=5, sg=1, min_count=1)

# Générer des embeddings
def generate_fasttext_sg_embedding(tokens):
    embeddings = [fasttext_sg_model.wv[word] for word in tokens if word in fasttext_sg_model.wv]
    return sum(embeddings) / len(embeddings) if embeddings else [0] * 100

data['Embedding'] = data['Filtered_Review'].apply(generate_fasttext_sg_embedding)


**2. Enregistrement des Embeddings dans un fichier excel**

In [6]:
from google.colab import files

def format_embedding(embedding):
    return '[' + ', '.join(f'{x:.10f}' for x in embedding) + ']'

# Création d'un DataFrame
output_data = data[['no', 'Hotel name', 'rating', 'user type', 'room type', 'nights', 'Normalized_Review', 'Sentiment']]
output_data['Embedding'] = data['Embedding'].apply(format_embedding)

print(output_data.head())

# Enregistrement des données dans un fichier Excel
output_filename = '/content/hotel_reviews_embeddings.xlsx'
output_data.to_excel(output_filename, index=False)

files.download(output_filename)

print("Le fichier Excel a été sauvegardé sous le nom 'hotel_reviews_embeddings.xlsx'.")

<ipython-input-6-dd8dd734d7f9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_data['Embedding'] = data['Embedding'].apply(format_embedding)


   no Hotel name  rating    user type                   room type  \
0   2    فندق 72       2  مسافر منفرد  غرفة ديلوكس مزدوجة أو توأم   
1   3    فندق 72       5          زوج  غرفة ديلوكس مزدوجة أو توأم   
2  16    فندق 72       5          زوج                           -   
3  20    فندق 72       1          زوج          غرفة قياسية مزدوجة   
4  23    فندق 72       4          زوج  غرفة ديلوكس مزدوجة أو توأم   

            nights                                  Normalized_Review  \
0  أقمت ليلة واحدة                      ممتاز النظافة والطاقم متعاون    
1  أقمت ليلة واحدة   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2      أقمت ليلتين  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3  أقمت ليلة واحدة   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4      أقمت ليلتين  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

  Sentiment                                          Embedding  
0  Negative  [-0.1103786975, 0.4333301485, 0.3309434056, -0...  
1  Positive  [-0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Le fichier Excel a été sauvegardé sous le nom 'hotel_reviews_embeddings.xlsx'.


# Étape 3: Entraînement d'un Classificateur

**1. Entraînement du Classificateur**

In [7]:
pip install torch scikit-learn

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Charger les données
file_path = '/content/hotel_reviews_embeddings.xlsx'
data = pd.read_excel(file_path)

# Préparer les données
X = data['Embedding']
y = data['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir le mapping des labels
label_mapping = {'Negative': 0, 'Positive': 1}
y_train = [label_mapping[label] for label in y_train]
y_test = [label_mapping[label] for label in y_test]

# Dataset PyTorch
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = [eval(emb) for emb in embeddings]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Définir le modèle LSTM
class SentimentLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(SentimentLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # LSTM retourne toutes les sorties et la dernière sortie cachée
        _, (hidden, _) = self.lstm(x)
        # On utilise la dernière couche cachée pour la classification
        hidden = hidden[-1]
        x = self.fc(hidden)
        return self.softmax(x)

# Hyperparamètres
hidden_dim = 128  # Nombre de dimensions dans les états cachés de LSTM
num_layers = 1  # Nombre de couches LSTM
input_dim = len(eval(X_train.iloc[0]))
output_dim = 2  # Binaire : Positif ou Négatif

# Instancier le modèle
model = SentimentLSTM(input_dim, hidden_dim, output_dim, num_layers)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for embeddings, labels in train_loader:
        embeddings = embeddings.unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1/10, Loss: 0.4997
Epoch 2/10, Loss: 0.4350
Epoch 3/10, Loss: 0.4294
Epoch 4/10, Loss: 0.4293
Epoch 5/10, Loss: 0.4270
Epoch 6/10, Loss: 0.4248
Epoch 7/10, Loss: 0.4273
Epoch 8/10, Loss: 0.4221
Epoch 9/10, Loss: 0.4221
Epoch 10/10, Loss: 0.4218


**2. Évaluation du model**

In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Évaluation du modèle
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings = embeddings.unsqueeze(1)
        outputs = model(embeddings)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

# Rapport de classification
print(classification_report(all_labels, all_preds, target_names=['Negative',  'Positive']))

              precision    recall  f1-score   support

    Negative       0.89      0.90      0.90      1149
    Positive       0.86      0.85      0.86       851

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



**3. Sauvegarde du Modèle**

In [11]:
from google.colab import drive
import os
import torch

drive.mount('/content/drive')

drive_model_dir = "/content/drive/My Drive/lstm_model"
os.makedirs(drive_model_dir, exist_ok=True)

# Sauvegarde du modèle
model_path = os.path.join(drive_model_dir, "Lstm_model.pth")
torch.save(model.state_dict(), model_path)

print(f"Le modèle LSTM est sauvegardé dans : {model_path}")


Mounted at /content/drive
Le modèle LSTM est sauvegardé dans : /content/drive/My Drive/lstm_model/Lstm_model.pth


# Étape 3: Test du model

In [12]:
from nltk.tokenize import word_tokenize
import torch
import numpy as np

def preprocess_text(text):
    """
    Tokenize le texte et filtre les stopwords.
    """
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in arabic_stopwords]
    return filtered_tokens

def generate_sentence_embedding(tokens, model):
    """
    Génère un embedding pour une phrase à partir des tokens.
    """
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)  # Moyenne des embeddings des mots
    else:
        return np.zeros(100)  # Embedding nul si aucun mot n'a un embedding

def predict_sentiment(model, tokens):
    """
    Prédit le sentiment à partir des embeddings générés.
    """
    embedding = generate_sentence_embedding(tokens, fasttext_sg_model)
    embedding_tensor = torch.tensor(embedding, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    # Passer les embeddings dans le modèle
    model.eval()
    with torch.no_grad():
        output = model(embedding_tensor)
        prediction = torch.argmax(output, dim=1).item()

    # Mapping des classes
    label_mapping = {0: "Negative", 1: "Positive"}
    return label_mapping[prediction]

# Boucle pour tester l'entrée utilisateur
while True:
    avis = input("Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : ")
    if avis.lower() == 'quit':
        print("Sortie du programme.")
        break

    # Prétraitement du texte
    tokens = preprocess_text(avis)

    # Prédiction du sentiment
    classe = predict_sentiment(model, tokens)
    print(f"L'avis est classé comme : {classe}")

Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : استمتعت بكل لحظة في هذا الفندق. الإفطار كان لذيذًا والخدمات ممتازة
L'avis est classé comme : Positive
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : الفندق قديم، يحتاج إلى تجديد
L'avis est classé comme : Negative
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : فندق جيد
L'avis est classé comme : Positive
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : سيء للغاية
L'avis est classé comme : Negative
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : التكييف ضعيف للغاي
L'avis est classé comme : Negative
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : موقع الفندق ممتاز، قريب من كل شيء
L'avis est classé comme : Positive
Entrez un avis en arabe pour prédire le sentiment (ou tapez 'quit' pour quitter) : الحمام ملوث، والخدمة سيئة
L'avis est cla